// Author: J Musinsky, December 2020
// National Ecological Observatory Network, Battelle

Resources
- https://developers.google.com/earth-engine/guides/python_install
- https://earthlab.colorado.edu/introduction-google-earth-engine-python-api
- https://courses.spatialthoughts.com/end-to-end-gee.html

In [1]:
import ee, geemap

In [ ]:
geemap.__version__

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
# Specify center location of SRE
geo = ee.Geometry.Point([-110.83549, 31.91068])

# Display Surface Directional Reflectance image

SRER_SDR2017 = ee.ImageCollection("projects/neon/DP3-30006-001_SDR").filterDate('2017-01-01', '2021-12-31').filterBounds(geo).first();
#SRER_SDR2017.uint8()
SRER_SDR2017rgb = SRER_SDR2017.updateMask(SRER_SDR2017.gte(0.0000)).uint16().select(['band053', 'band035', 'band019']);
visParams = {min:0,max:255};
#Map.addLayer(SRER_SDR2017mask, trueColorVis, 'True Color');
Map.addLayer(SRER_SDR2017, visParams, 'SRER Reflectance 2017');
#Map.addLayer(SRER_SDR2017rgb, visParams, 'SRER RGB 2017');
Map.setCenter(-110.83549, 31.91068, 10);

In [7]:
Map.set_plot_options(plot_type='line')

In [ ]:
# Display all metadata.
print('All metadata:', SRER_SDR2017);

In [ ]:
bandNames = SRER_SDR2017.bandNames();
print('Band names:', bandNames);  # ee.List of band names

In [ ]:
image_max = SRER_SDR2017mask.reduceRegion({
  reducer: ee.Reducer.max(),
  geometry: ,
});
maxXT = image_mask.get('band053').getInfo()
print(maxXT)

In [ ]:
# Define a RasterSymbolizer element with '_enhance_' for a placeholder.
template_sld = \
  '<RasterSymbolizer>' + \
    '<ContrastEnhancement><_enhance_/></ContrastEnhancement>' + \
    '<ChannelSelection>' + \
      '<RedChannel>' + \
        '<SourceChannelName>band053</SourceChannelName>' + \
      '</RedChannel>' + \
      '<GreenChannel>' + \
        '<SourceChannelName>band035</SourceChannelName>' + \
      '</GreenChannel>' + \
      '<BlueChannel>' + \
        '<SourceChannelName>band019</SourceChannelName>' + \
      '</BlueChannel>' + \
    '</ChannelSelection>' + \
  '</RasterSymbolizer>'

# Get SLDs with different enhancements.
equalize_sld = template_sld.replace('_enhance_', 'Histogram')
normalize_sld = template_sld.replace('_enhance_', 'Normalize')

# Display the results.
# Map.centerObject(image, 10)
# Map.addLayer(image, {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 15000}, 'Linear')
# Map.addLayer(SRER_SDR2017mask.sldStyle(equalize_sld), {}, 'Equalized')
Map.addLayer(SRER_SDR2017mask.sldStyle(normalize_sld), {}, 'Normalized')

In [ ]:
image = ee.ImageCollection('projects/neon/DP3-30006-001_SDR').filterBounds('SRER').filterDate('2017-01-01', '2017-12-31').filterBounds('SRER').first()

In [ ]:
# Add AOP flight boxes asset as Feature Collection 
AOPsites = ee.FeatureCollection('projects/neon/AOPflightboxes/P1flightboxesAllSitesV2_GEEsubset')
names = AOPsites.aggregate_array('siteID')

In [ ]:
# display each NIS image in the collection
def addNISImage(image):
# get the system:id and convert to string; note this is actually an object on the server
    print(fileName)
    id = image.id
    im = ee.Image(image.id);
    sysID_serverObj = ee.String(image.get("system:id"));
    # getInfo() converts to string on the server
    sysID_serverStr = sysID_serverObj.getInfo()
    # truncate the string to show only the fileName
    fileName = sysID_serverStr.slice(46,100); 
    # mask out non-data values
    image3 = image.updateMask(image.gte(0.0000)).select(['band053', 'band035', 'band019']);
    Map.addLayer(image3, {min:.5, max:10}, fileName)
    # add NDVI layers
    ndviViz={min:-0.5, max:1,palette:['blue', 'brown', 'white']}
    NISndvi=image.normalizedDifference(["band097", "band055"]).rename("ndvi")
    Map.addLayer(NISndvi, ndviViz, "NDVI "+fileName,0)

In [ ]:
NISimages = ee.ImageCollection('projects/neon/DP3-30006-001_SDR').filterBounds(mySite)

In [ ]:
# Labeling wavelengths with wavelength and index
wavelengths = ee.List.sequence(381, 2510, 5).getInfo()
bands_no =  ee.List.sequence(1, 426).getInfo() 
# l8wavelengths = [440, 480, 560, 650, 860, 1610, 2200];

In [ ]:
# Create a panel to hold the text box.
textPanel = ui.Panel();
textPanel.style().set({
  width: '370px',
  position: 'top-center'
});

Map.add(textPanel);
# Create and print the text box with "refresh" text
newText=(ui.Textbox({}, 'Refresh browser window (F5) to select new NEON site', {}, {}, {width:'340px', fontWeight: 'bold'}))
textPanel.add(newText)

```
// Add AOP flight boxes asset as Feature Collection
var AOPsites = ee.FeatureCollection('projects/neon/AOPflightboxes/P1flightboxesAllSitesV2_GEEsubset')

var names = AOPsites.aggregate_array('siteID')

function onNameChanged(siteID) {
  print('Update map for ' + siteID)
}

names.evaluate(function(names) {
  var currentName = ui.Select({
    items: names, 
    placeholder: 'Select NEON Site', 
    onChange: redraw,
  });
  var panel = ui.Panel([currentName]);
   Map.widgets().add(panel);
});

var layers = [
  ui.Map.Layer(AOPsites),
];

// The new value is passed as a parameter to the select onChange callback.
function redraw(names) {
    // Filter the collection to contain only the selected feature
    var mySite = AOPsites.filterMetadata("siteID","equals", names)
    var siteName = AOPsites.filterMetadata("siteID","equals", names)
    var FBsiteID = ee.String(names).getInfo()

// Helper Function to remove all previous layers of the map
    function removeLayersFromMap(layer){
      Map.remove(layer)
    }

var getQABits = function(image, start, end, newName) {
    // Compute the bits we need to extract.
    var pattern = 0;
    for (var i = start; i <= end; i++) {
      pattern += Math.pow(2, i);
    }
    // Return a single band image of the extracted QA bits, giving the band
    // a new name.
    return image.select([0], [newName])
                  .bitwiseAnd(pattern)
                  .rightShift(start);
};

// Define a function to mask out poor quality pixels (bits 0 and 1) in MODIS
var mask= function(image) {
//Select the QA band.
  var QA = image.select('SummaryQA');
  var cloud = getQABits(QA, 0, 1, 'VI quality')
                        .expression("b(0) == 2 || b(0) == 3");
                        //.expression("b(0) == 1 || b(0) == 2 || b(0) == 3");
 // Return an image masking out poor quality (cloudy) areas
return image.updateMask(cloud.not())//.internalCloud.eq(0));
};


    Map.layers().map(removeLayersFromMap)  // Map the function to the Map.layers() object
    Map.centerObject(mySite,12)

var geo = mySite

var NISimages = ee.ImageCollection('projects/neon/DP3-30006-001_SDR')
  .filterBounds(mySite)

var DEMimages =  ee.ImageCollection('projects/neon/DP3-30024-001_DEM')
  .filterBounds(mySite)
  
var RGBimages = ee.ImageCollection('projects/neon/DP3-30010-001_RGB')
  .filterBounds(mySite)

var NIS2017 = ee.ImageCollection('projects/neon/DP3-30006-001_SDR')
  //.filterDate('2017-01-01', '2017-12-31')
  .filterBounds(mySite)
  .first();

var NIS2018 = ee.ImageCollection('projects/neon/DP3-30006-001_SDR')
  .filterDate('2018-01-01', '2019-12-31')
  .filterBounds(mySite)
  .first();

var DTM2017 = ee.ImageCollection('projects/neon/DP3-30024-001_DEM')
  .filterDate('2017-01-01', '2017-12-31')
  .filterBounds(mySite)
  .filterMetadata('Type', 'equals', 'DTM')
  .first();
  
var DTM2017percentClip = DTM2017.reduceRegion({
  reducer: ee.Reducer.percentile([2, 98]),
  scale: 10,
  maxPixels: 3e7
});

// Labeling Wavelengths with wavelength and index
var wavelengths = ee.List.sequence(381, 2510, 5).getInfo()
var bands_no =  ee.List.sequence(1, 426).getInfo() 
var l8wavelengths = [440, 480, 560, 650, 860, 1610, 2200];

Map.clear();  

// Create a panel to hold the text box.
var textPanel = ui.Panel();
textPanel.style().set({
  width: '370px',
  position: 'top-center'
});

Map.add(textPanel);
// Create and print the text box with "refresh" text
var newText=(ui.Textbox({}, 'Refresh browser window (F5) to select new NEON site', {}, {}, {width:'340px', fontWeight: 'bold'}))
textPanel.add(newText)

// Print the regional 2nd and 98th percentile elevation values. Get the
// dictionary keys and use them to get the values for each percentile summary.
var keys = DTM2017percentClip.keys();
var minserver = ee.Number(DTM2017percentClip.get(keys.get(0))).round()
var maxserver = ee.Number(DTM2017percentClip.get(keys.get(1))).round()
var minclient = minserver.getInfo()
var maxclient = maxserver.getInfo()

// display each DEM image in the collection
function addDTMImage(image) { // display each image in collection
// get the system:id and convert to string
// note this is actually an object on the server
print(fileName)
  var id = image.id
  var image1 = ee.Image(image.id);
  var sysID_serverObj = ee.String(image1.get("system:id"));
  // getInfo() converts to string on the server
  var sysID_serverStr = sysID_serverObj.getInfo()
  // cut the string down for the fileName needed
  var fileName = sysID_serverStr.slice(46,100); 
  // mask out non-data values
  var image2 = image1.updateMask(image1.gte(0.0000));
  Map.addLayer(image2, {min:minclient, max:maxclient}, fileName, 0)
// print(image)
}

DEMimages.evaluate(function(DEMimages) {  // use map on client-side
  DEMimages.features.map(addDTMImage);
})

// display each RGB image in the collection
function addRGBImage(image) { // display each image in collection
// get the system:id and convert to string
// note this is actually an object on the server
print(fileName)
  var id = image.id
  var image4 = ee.Image(image.id);
  var sysID_serverObj = ee.String(image4.get("system:id"));
  // getInfo() converts to string on the server
  var sysID_serverStr = sysID_serverObj.getInfo()
  // cut the string down for the fileName needed
  var fileName = sysID_serverStr.slice(46,100); 
  // mask out non-data values
  var image5 = image4.updateMask(image4.gt(0.0000)).select(['band001', 'band002', 'band003'])
  Map.addLayer(image5, {min:50, max:250}, fileName, 0)
// print(image)
}

RGBimages.evaluate(function(RGBimages) {  // use map on client-side
  RGBimages.features.map(addRGBImage);
})

// display each NIS image in the collection
function addNISImage(image) { 
// get the system:id and convert to string; note this is actually an object on the server
print(fileName)
  var id = image.id
  var image = ee.Image(image.id);
  var sysID_serverObj = ee.String(image.get("system:id"));
  // getInfo() converts to string on the server
  var sysID_serverStr = sysID_serverObj.getInfo()
  // truncate the string to show only the fileName
  var fileName = sysID_serverStr.slice(46,100); 
  // mask out non-data values
  var image3 = image.updateMask(image.gte(0.0000)).select(['band053', 'band035', 'band019']);
  Map.addLayer(image3, {min:.5, max:10}, fileName)
 //add NDVI layers
  var ndviViz={min:-0.5, max:1,palette:['blue', 'brown', 'white']}
  var NISndvi=image.normalizedDifference(["band097", "band055"]).rename("ndvi")
  Map.addLayer(NISndvi, ndviViz, "NDVI "+fileName,0)
}

NISimages.evaluate(function(NISimages) {  // use map on client-side
  NISimages.features.map(addNISImage);
})

Map.addLayer(AOPsites.style({width: 2, color: "blue", fillColor: "#00000000"}),{},"AOP Priority 1 Flight Box")

// Note that the input to simpleComposite is raw data.
var l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1');
// The asFloat parameter gives floating-point TOA output instead of
// the UINT8 outputs of the default simpleComposite().
var composite = ee.Algorithms.Landsat.simpleComposite({
  collection: l8.filterDate('2019-05-01', '2019-09-01'),
  asFloat: false //true
});

// Display a composite with a band combination chosen from:
//Map.addLayer(composite, {bands: ['B6', 'B5', 'B4'], max: [0.3, 0.4, 0.3]}, 'Landsat 8 2019', 0);
Map.addLayer(composite, {bands: ['B6', 'B5', 'B4'], max: [100, 110, 100]}, 'Landsat 8 2019', 0);


//load the MODIS TERRA image collection
var modisEVI = ee.ImageCollection('MODIS/006/MOD13Q1') //MOD13Q1.006 Terra Vegetation Indices 16-Day Global 250m
    .filter(ee.Filter.dayOfYear(0, 365))
    .filterDate('2015-01-01', '2019-12-31')
    .filterBounds(mySite)

// Create the title label.
var title = ui.Label('Click on map to view bands');
title.style().set('position', 'bottom-center');
Map.add(title);

// Create a panel to hold the first chart.
var panel = ui.Panel();
panel.style().set({
  width: '600px',
  height: '300px',
  position: 'top-left'
});
Map.add(panel);
Map.style().set('cursor', 'crosshair');

// Create a panel to hold the second chart.
// var panel2 = ui.Panel();
// panel2.style().set({
//   width: '600px',
//   height: '300px',
//   position: 'bottom-left'
// });
// Map.add(panel2);

// Create a panel to hold the third chart (Landsat 8).
// var panel3 = ui.Panel();
// panel3.style().set({
//   width: '600px',
//   height: '300px',
//   position: 'bottom-right'
// });
// Map.add(panel3);

// Create a panel to hold the fourth chart (MODIS EVI).
// var panel4 = ui.Panel();
// panel4.style().set({
//   width: '600px',
//   height: '300px',
//   position: 'top-right'
// });
// Map.add(panel4);

// Register a function to draw a chart when a user clicks on the map.
Map.onClick(function(coords) {
// var NIS2017list = NIS2017.reduceRegion(ee.Reducer.toList())
// var NIS2018list = NIS2018.reduceRegion(ee.Reducer.toList())
// var y1 = ee.Array(NIS2017list)
// var y2 = ee.Array(NIS2018list)
// var yValues = ee.Array.cat([y1, y2], 1)
// var xValues = NIS2017.bandNames()
// print(yValues)
// print(xValues)
  panel.clear();
  var point = ee.Geometry.Point(coords.lon, coords.lat);
  var chart = ui.Chart.image.regions(NIS2017, point, null, 1, 'λ (nm)', wavelengths);
//  var chart = ui.Chart.image.regions(yValues, point, null, 30)
  chart.setSeriesNames(['Spectral Values'])
  .setOptions({title: 'NEON ' + FBsiteID + ': 1st Date',
    hAxis: {title: 'Wavelengths (nm)', gridlines: { count: 5 }},
    vAxis: {title: 'Reflectance'}
  });
    // Create or update the location label (the second widget in the panel)
  var location = 'Longitude: ' + coords.lon.toFixed(2) + ' ' +
                 'Latitude: ' + coords.lat.toFixed(2);
  panel.widgets().set(1, ui.Label(location));
  panel.add(chart);

  // panel2.clear();
  // var point2 = ee.Geometry.Point(coords.lon, coords.lat);
  // var chart2 = ui.Chart.image.regions(NIS2018, point2, null,  1, 'λ (nm)', wavelengths);
  // chart2.setSeriesNames(['Spectral Values'])
  // .setOptions({title: 'NEON  ' + FBsiteID + ': 2nd Date',
  //   hAxis: {title: 'Wavelengths (nm)'},
  //   vAxis: {title: 'Reflectance'},
  // });

  // panel2.widgets().set(1, ui.Label(location));
  // panel2.add(chart2);
  
  // panel3.clear();
  // var point3 = ee.Geometry.Point(coords.lon, coords.lat);
  // var chart3 = ui.Chart.image.regions(composite.select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'), point3, null, 30, 'λ (nm)', l8wavelengths);
  // chart3.setSeriesNames(['Spectral Values'])
  // .setOptions({title: 'Landsat 8 2019 Composite',
  //   hAxis: {title: 'Wavelengths (nm)', gridlines: { count: 5 }},
  //   vAxis: {title: 'Scaled TOA Reflectance'},
  // });

  // panel3.widgets().set(1, ui.Label(location));
  // panel3.add(chart3);


// panel4.clear();
//   var point4 = ee.Geometry.Point(coords.lon, coords.lat);
//   var chart4 = ui.Chart.image.doySeriesByYear(
//     modisEVI, 'EVI', point4, ee.Reducer.mean(), 250);
//   // ui.Chart.image.regions(composite.select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'), point3, null, 30);
//   chart4.setSeriesNames(['2015'])
//   .setOptions({title: 'MODIS EVI',
//     hAxis: {title: 'DOY'},
//     vAxis: {title: 'Scaled EVI'},
//   });
//   panel4.widgets().set(1, ui.Label(location));
//   panel4.add(chart4);

// print(modisEVI)    
// // Display the chart
// print(series);

  
  
});

}

Map.addLayer(AOPsites.style({width: 3, color: "blue", fillColor: "#00000000"}),{},"AOP Flight Boxes")

Map.centerObject(AOPsites,4.5)
```